# Prediction

Deployment of kserve will take few minutes to comes to ready state. In just a short span of time, it will transition into a ready state, providing the tools you need at your fingertips.

Now, let us lay the groundwork for what comes next. The first crucial step you will need to undertake is securing an access token. This token serves as your personal key, allowing you to invoke the service.

In [ ]:
import warnings
import requests

warnings.filterwarnings('ignore')

In [ ]:
# Add heading
heading = widgets.HTML("<h2>User Credentials</h2>")
display(heading)

ezaf_env_input = widgets.Text(description='EZAF Env:')
namespace_input = widgets.Text(description='Namespace:')
deployment_input = widgets.Text(value="bike-sharing", description='Deployment:')
username_input = widgets.Text(description='Username:')
password_input = widgets.Password(description='Password:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

ezaf_env = None
namespace = None
username = None
password = None

def submit_button_clicked(b):
    global ezaf_env, namespace, deployment, username, password
    ezaf_env = ezaf_env_input.value
    namespace = namespace_input.value
    deployment = deployment_input.value
    username = username_input.value
    password = password_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(ezaf_env_input, namespace_input, deployment_input,
        username_input, password_input, submit_button, success_message)

In [ ]:
# Provide your EZAF subdomain here:
EZAF_ENV = ezaf_env
token_url = f"https://keycloak.{EZAF_ENV}.com/realms/UA/protocol/openid-connect/token"

data = {
    "username" : username,
    "password" : password,
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

Finally, you are ready to send a POST request to your InferenceService, providing the features in the body of the request. In the response, you'll get the prediction of the model.

In [ ]:
NAMESPACE = namespace
DEPLOYMENT_NAME = deployment
MODEL_NAME = DEPLOYMENT_NAME
SVC = f'{DEPLOYMENT_NAME}-predictor-default.{NAMESPACE}.{EZAF_ENV}.com'
URL = f"https://{SVC}/v2/models/{MODEL_NAME}/infer"

names = ['season', 'year', 'month', 'hour_of_day', 'is_holiday', 'weekday', 'is_workingday', 
         'weather_situation', 'temperature', 'feels_like_temperature', 'humidity', 'windspeed']

input_data = [
    [1, 2, 1, 0, 0, 6, 0, 1, 0.24, 0.2879, 0.81, 0.0000],
    [1, 5, 1, 0, 0, 6, 1, 1, 0.24, 0.2879, 0.81, 0.0000]
]

inputs = {
  "inputs": [
    {
      "name": "ndarray",
      "shape": [2, 12],
      "datatype": "FP32",
      "data": input_data
    }
  ]
}

headers = {"Authorization": f"Bearer {token}"}

response = requests.post(URL, json=inputs, headers=headers, verify=False)

print(response.reason)

output = response.json()['outputs'][0]['data']

print("\nRendted Bikes Per Hour:\n")
for item, out in zip(input_data, output):
    input_dict = dict(zip(names,item))
    print(f"Input Data: {input_dict} \nBike Per Hour: {out}\n")